In [1]:
#Importing relevant libraries
import numpy as np
import pandas as pd
import torch
from collections import Counter

import random
import os
import re
import json

In [2]:
def get_provo_data(input_data):
    """A function that takes all .json files we created with info for the Provo Corpus
    and merges it into one dictionary"""
    
    #We merge all information in one dictionary
    # Each data point corresponds to all the information relevant to us for a given context in Provo Corpus
    joint_dict = {}
    
    count = 0
    for filename in input_data:
        f = open(filename)
        data = json.load(f)
        f.close()

        for text_id in data.keys():
            if (int(text_id) > 0) & (int(text_id) <= 55):
                for word_num in data[text_id].keys():
                    joint_dict[count] = data[text_id][word_num]
                    joint_dict[count]['original_positioning'] = {'text_id':text_id, 'word_num':word_num}
                
                    count = count + 1

    return joint_dict

In [3]:
os.chdir(os.path.join(os.getcwd(), '/TVD/GPT2/Generations'))

input_data = ['Paragraphs-1-1.json', 'Paragraphs-2-2.json', 'Paragraphs-3-3.json',
    'Paragraphs-4-4.json', 'Paragraphs-5-9.json', 'Paragraphs-10-14.json', 
    'Paragraphs-15-19.json', 'Paragraphs-20-24.json', 'Paragraphs-25-29.json',
    'Paragraphs-30-34.json', 'Paragraphs-35-39.json', 'Paragraphs-40-44.json', 
    'Paragraphs-45-47.json', 'Paragraphs-48-50.json', 'Paragraphs-51-53.json',
     'Paragraphs-54-55.json']

d = get_provo_data(input_data)


In [4]:
os.chdir(os.path.join(os.getcwd(), '/TVD/GPT2_Temp/Generations'))

input_data = ['Paragraphs_temp-1-1.json', 'Paragraphs_temp-2-4.json', 
    'Paragraphs_temp-5-7.json', 'Paragraphs_temp-8-17.json', 'Paragraphs_temp-18-29.json', 
    'Paragraphs_temp-30-42.json', 'Paragraphs_temp-43-55.json']

d_temp = get_provo_data(input_data)

In [6]:
for key in d.keys():
    human_samp= [[x['pred']]*int(x['count']) for x in d[key]['human']]
    human_samp = [item for sublist in human_samp for item in sublist]
    original_word = d[key]['original']['pred']
    gpt2_samp = [x for x in d[key]['ancestral_samples']]
    gpt2_samp = [[x['pred']]*int(x['count']) for x in gpt2_samp]
    gpt2_samp = [item.lower() for sublist in gpt2_samp for item in sublist]


In [7]:
human_samples = []
original_corpus_words = []
gpt2_samples = []

for key in d.keys():
    human_samp= [[x['pred']]*int(x['count']) for x in d[key]['human']]
    human_samp = [item for sublist in human_samp for item in sublist]
    original_word = d[key]['original']['pred']
    gpt2_samp = [x for x in d[key]['ancestral_samples']]
    gpt2_samp = [[x['pred']]*int(x['count']) for x in gpt2_samp]
    gpt2_samp = [item.lower() for sublist in gpt2_samp for item in sublist]

    human_samples.append(human_samp)
    gpt2_samples.append(gpt2_samp)
    original_corpus_words.append([original_word])

In [8]:
human_samples_temp = []
original_corpus_words_temp = []
gpt2_samples_temp = []

for key in d_temp.keys():
    human_samp= [[x['pred']]*int(x['count']) for x in d_temp[key]['human']]
    human_samp = [item for sublist in human_samp for item in sublist]
    original_word = d_temp[key]['original']['pred']
    gpt2_samp = [x for x in d_temp[key]['ancestral_samples']]
    gpt2_samp = [[x['pred']]*int(x['count']) for x in gpt2_samp]
    gpt2_samp = [item.lower() for sublist in gpt2_samp for item in sublist]

    human_samples_temp.append(human_samp)
    gpt2_samples_temp.append(gpt2_samp)
    original_corpus_words_temp.append([original_word])

In [9]:
def sample_oracle_without_replacement_disjoint_groups(words, seed, N = 20):
    """We create two disjoint subsets of the human distribution by sampling without replacement from
    the human distribution (the two disjoing subsets can be comprised by either 10 or 20 samples"""
    #Create a list with all human answers in a flattened out list ['are', 'are', 'they', ..., 'one']
    random.seed(seed)

    #if the length of the list is odd, we remove one element at random to make the list even,
    #since we want the two disjoint subsets to be of equal length
    if (len(words) % 2 == 1): 
        remove_word = random.sample(words, 1)
        words.remove(remove_word[0])

    #We sample the words that will belong in the first subset and create the second subset by removing
    #from the full word list the ones sampled in the first subset
    subset1 = random.sample(words, N)
    if N == 20:
        subset2 = words.copy()
        for item in subset1:
            subset2.remove(item)
    elif N == 10:
        subset_left = words.copy()
        for item in subset1:
            subset_left.remove(item)
        subset2 = random.sample(subset_left, N)
        
    return subset1, subset2

In [10]:
def get_estimator_unbiased(words):
    """For each data point we compute the estimator where the words belong to the unbiased distribution"""
    #Check for failed to generate full-word samples and remove those
    fail = [d for d in words if d == 'Failed to generate word'.lower()]
    if len(fail) > 0:
        words= [x for x in words if x != 'Failed to generate word']
    
    #print(words)
    words = [x for x in words if str(x) != 'nan']
    words = [word.lower() for word in words]
    dict_words = dict(Counter(words))
    support = list(dict_words.keys())
    counts = list(dict_words.values())
    probs = torch.Tensor([x/sum(counts) for x in counts])

    return support, probs

In [11]:
def get_ece_data( assess_words, assess_probs, gold_label_words, gold_label_probs):
    """Considering the estimator distribution, we obtain the word (and its confidence) with the maximum 
    probability. For computing accuracy we consider if this word matches the true label (which we consider for 
    both the cases where they are either the original text word and human majority word"""
    gold_label_probs = gold_label_probs.tolist()
    gold_label_majority_word = gold_label_words[gold_label_probs.index(max(gold_label_probs))]
    
    p_max_word = assess_words[torch.argmax(assess_probs).item()]
    human_maj = (torch.max(assess_probs).item(), int(p_max_word == gold_label_majority_word)) 

    return human_maj

In [12]:
def compute_TVD(probs1, probs2):
    tvd = torch.sum(torch.abs(probs1 - probs2))/2
    return tvd.item()

In [13]:
def compute_entropy(probs):
    #For zero probability values of p in p log p, the contribution to entropy is 0, hence we take only
    #non zero p values into account
    non_zero_probs = probs[probs > 0]
    entropy_probs = - torch.sum(torch.multiply(non_zero_probs, torch.log(non_zero_probs)))

    return entropy_probs.tolist()

In [14]:
def get_tvd_per_instance_for_unbiased_est_dist_and_oracle(model_words, model_probs, oracle_words, oracle_probs):
        """Given the distribution (from the model), we retrieve the human distribution for the same words,
        and then compute TVD for the instance level"""

        #We know that the items of the model distribution and the oracle distribution are not currently aligned
        #Thus, before computing the TVD between them we first need to align the sample space and probabilities between
        #the two distributions
        human_probs = []
        
        list_model_probs = model_probs.tolist()
        list_words = model_words.copy()

        #For the unbiased distributions, the sampled words may not necessarily include all human words. Hence,
        # before creating the human distribution, we add to the model one the ones that are missing with a respective 
        # probability of zero
        list_missing = list(set(oracle_words) - set(list_words)) #set of human words that are not in the model distribution words

        for missing_word in list_missing:
            list_words.append(missing_word)
            list_model_probs.append(0)
        
        #Similarly to the biased dist., we iterate over all words and the human dist. probabilities are either the retrieved
        #probability from the oracle dist. or zero
        for word in list_words:
            try:
                index_word = oracle_words.index(word)
                human_probs.append(oracle_probs[index_word].item())
            except:
                human_probs.append(0)

        tvd = compute_TVD(torch.Tensor(human_probs), torch.Tensor(list_model_probs))
        return(tvd)

In [15]:
tvd_gpt2_human = []
tvd_gpt2_oracle = []
tvd_oracle2_human = []
tvd_oracle1_human = []

ece_gpt2_human_maj = []
ece_gpt2_corpus_word = []
ece_gpt2_oracle_maj = {}

entropy_gpt2 = []
entropy_human = []
entropy_oracle = []

for k in range(20):
    ece_gpt2_oracle_maj[k] = []

for i in range(len(gpt2_samples) - 1):
    human_support, human_probs = get_estimator_unbiased(human_samples[i])
    gpt2_support, gpt2_probs = get_estimator_unbiased(gpt2_samples[i])
    corpus_support = original_corpus_words[i]
    corpus_probs = torch.Tensor([1])
    oracle_1, oracle_2 = sample_oracle_without_replacement_disjoint_groups(human_samples[i], seed =1)
    oracle1_support, oracle1_probs = get_estimator_unbiased(oracle_1)
    oracle2_support, oracle2_probs = get_estimator_unbiased(oracle_2)

    tvd_gpt2_human.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(gpt2_support, gpt2_probs, human_support, human_probs ))
    tvd_gpt2_oracle.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(gpt2_support, gpt2_probs, oracle1_support, oracle1_probs ))
    tvd_oracle2_human.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(oracle2_support, oracle2_probs, human_support, human_probs ))
    tvd_oracle1_human.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(oracle1_support, oracle1_probs, human_support, human_probs ))

    entropy_gpt2.append(compute_entropy(gpt2_probs))
    entropy_human.append(compute_entropy(human_probs))
    entropy_oracle.append(compute_entropy(oracle1_probs))

    ece_gpt2_human_maj.append(get_ece_data( gpt2_support, gpt2_probs, human_support, human_probs))
    ece_gpt2_corpus_word.append(get_ece_data( gpt2_support, gpt2_probs, corpus_support, corpus_probs))
    
    for k in range(20):
        oracle_1, oracle_2 = sample_oracle_without_replacement_disjoint_groups(human_samples[i], k)
        oracle_support, oracle_probs = get_estimator_unbiased(oracle_1)
        ece_gpt2_oracle_maj[k].append(get_ece_data( gpt2_support, gpt2_probs, oracle_support, oracle_probs))


In [16]:
tvd_gpt2_human_temp = []
tvd_gpt2_oracle_temp = []
tvd_oracle2_human_temp = []
tvd_oracle1_human_temp = []

ece_gpt2_human_maj_temp = []
ece_gpt2_corpus_word_temp = []
ece_gpt2_oracle_maj_temp = {}

entropy_gpt2_temp = []
entropy_human_temp = []
entropy_oracle_temp = []

for k in range(20):
    ece_gpt2_oracle_maj_temp[k] = []

for i in range(len(gpt2_samples_temp) - 1):
    human_support, human_probs = get_estimator_unbiased(human_samples_temp[i])
    gpt2_support, gpt2_probs = get_estimator_unbiased(gpt2_samples_temp[i])
    corpus_support = original_corpus_words_temp[i]
    corpus_probs = torch.Tensor([1])
    oracle_1, oracle_2 = sample_oracle_without_replacement_disjoint_groups(human_samples_temp[i], seed =1)
    oracle1_support, oracle1_probs = get_estimator_unbiased(oracle_1)
    oracle2_support, oracle2_probs = get_estimator_unbiased(oracle_2)

    tvd_gpt2_human_temp.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(gpt2_support, gpt2_probs, human_support, human_probs ))
    tvd_gpt2_oracle_temp.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(gpt2_support, gpt2_probs, oracle1_support, oracle1_probs ))
    tvd_oracle2_human_temp.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(oracle2_support, oracle2_probs, human_support, human_probs ))
    tvd_oracle1_human_temp.append(get_tvd_per_instance_for_unbiased_est_dist_and_oracle(oracle1_support, oracle1_probs, human_support, human_probs ))

    entropy_gpt2_temp.append(compute_entropy(gpt2_probs))
    entropy_human_temp.append(compute_entropy(human_probs))
    entropy_oracle_temp.append(compute_entropy(oracle1_probs))

    ece_gpt2_human_maj_temp.append(get_ece_data( gpt2_support, gpt2_probs, human_support, human_probs))
    ece_gpt2_corpus_word_temp.append(get_ece_data( gpt2_support, gpt2_probs, corpus_support, corpus_probs))
    
    for k in range(20):
        oracle_1, oracle_2 = sample_oracle_without_replacement_disjoint_groups(human_samples_temp[i], k)
        oracle_support, oracle_probs = get_estimator_unbiased(oracle_1)
        ece_gpt2_oracle_maj_temp[k].append(get_ece_data( gpt2_support, gpt2_probs, oracle_support, oracle_probs))

In [17]:
#Expected TVD between Humans and GPT2 distributions
np.mean(tvd_gpt2_human) 

0.6428336816353062

In [19]:
#Expected TVD between Humans and GPT2 (with temp=0.5) distributions
np.mean(tvd_gpt2_human_temp) 

0.6066550797091455

In [20]:
def calculate_ECE(conf_acc, ece_bins = 10):
    """Function that given a list of tuples including the confidence of each prediction and if it matches the
    true label computes the ECE. To do that, we split the confidence space (0,1) in bins, separate predictions
    according to the bins, calculate the average confidence per bin and the accuracy per bin and take their weighted
    average."""
    bins = ece_bins
    conf_acc_array = np.array(conf_acc)
    N = conf_acc_array.shape[0]
        
    sum_bin = 0
    for i in np.arange(0, 1, 1/bins):
        #getting all points which belong to the relevant bin - given their 
        bin_contents = conf_acc_array[np.where((conf_acc_array[:,0] >= i) & (conf_acc_array[:,0] < (i + 1/bins)))]
        n_bin = bin_contents[:,0].shape[0]
        if n_bin > 0: #if the bin is non empty
            avg_conf = np.sum(bin_contents[:,0]) / n_bin
            acc = np.sum(bin_contents[:,1]) / n_bin
            sum_bin = sum_bin + abs(avg_conf - acc) * n_bin / N
        
    ece_val = sum_bin
    return(ece_val) 

In [21]:
#Get ECE for GPT2 (with temp = 0.5) using the human majority
calculate_ECE(ece_gpt2_human_maj_temp)

0.13425167501999075

In [22]:
#Get ECE for GPT2 (with temp = 0.5) using the corpus word
calculate_ECE(ece_gpt2_corpus_word_temp)

0.3498138492567741

In [23]:
#Get average ECE and respective standard deviation for GPT2 (with temp = 0.5) using the oracle majority 
#accross different runs
ece_subsamples = []
for i in range(20):
    ece_subsamples.append(calculate_ECE(ece_gpt2_oracle_maj[i]))

In [23]:
np.std(ece_subsamples)

0.015809317792561967

In [24]:
np.mean(ece_subsamples)

0.18534326110262636

In [26]:
results = {}

results['tvd_bloom_human'] = tvd_gpt2_human_temp
results['tvd_oracle1_human'] = tvd_oracle1_human_temp
results['tvd_oracle2_human'] = tvd_oracle2_human_temp
results['tvd_bloom_oracle'] = tvd_gpt2_oracle_temp

results['entropy_bloom'] = entropy_gpt2_temp
results['entropy_human'] = entropy_human_temp
results['entropy_oracle'] = entropy_oracle_temp

In [27]:
json_string = json.dumps(results)

with open('TVD_GPT2_temp.json', 'w') as outfile:
    outfile.write(json_string)